# Logo detection with pre-trained CenterNet (hourglass104) model.

## Prepare the working environment

In [ ]:
import os

ROOT_PATH = os.path.join(os.sep, 'content')
TFOD_DIR_NAME = 'tf_object_detection'
PRETRAINED_MODEL_NAME = 'centernet_hg104_512x512_coco17_tpu-8'

paths = {
    'DRIVE': os.path.join(ROOT_PATH, 'drive'),
    'DATASET': os.path.join(ROOT_PATH, 'dataset'),
    'LOGODET_3K': os.path.join(ROOT_PATH, 'dataset', 'LogoDet-3K'),
    'LOGODET_3K_FLAT_SUBSET': os.path.join(ROOT_PATH, 'dataset', 'LogoDet_3K_flat_subset'),
    'TRAIN_SET': os.path.join(ROOT_PATH, 'dataset', 'train'),
    'TEST_SET': os.path.join(ROOT_PATH, 'dataset', 'test'),
    'ANNOTATIONS': os.path.join(ROOT_PATH, 'annotations'),
    'PRETRAINED_MODEL': os.path.join(ROOT_PATH, PRETRAINED_MODEL_NAME),
    'OBJECT_DETECTION': os.path.join(ROOT_PATH, TFOD_DIR_NAME, 'research', 'object_detection'),
}

files = {
    'LOGODET_3K': os.path.join(paths['DRIVE'], 'MyDrive', 'LogoDet-3K.zip'), # Edit this path to be the location of your dataset
    'LABEL_MAP': os.path.join(paths['ANNOTATIONS'], 'label_map.pbtxt'),
    'TRAINING_SCRIPT': os.path.join(paths['OBJECT_DETECTION'], 'model_main_tf2.py'),
    'PIPELINE_CONFIG': os.path.join(ROOT_PATH, PRETRAINED_MODEL_NAME, 'pipeline.config'),
}

if not os.path.isdir(paths['ANNOTATIONS']):
    os.mkdir(paths['ANNOTATIONS'])
if not os.path.isdir(paths['DATASET']):
    os.mkdir(paths['DATASET'])
if not os.path.isdir(paths['LOGODET_3K_FLAT_SUBSET']):
    os.mkdir(paths['LOGODET_3K_FLAT_SUBSET'])

In [ ]:
# Install a package that splits the dataset in train and test sets
!wget https://github.com/paul2048/annotated-images/archive/refs/heads/master.zip
!unzip {ROOT_PATH}/master.zip
!pip install {ROOT_PATH}/annotated-images-master
!rm -rf annotated-images-master master.zip
!rm -rf sample_data
# Clone the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git
!mv models {ROOT_PATH}/{TFOD_DIR_NAME}

--2022-05-08 12:06:06--  https://github.com/paul2048/annotated-images/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/paul2048/annotated-images/zip/refs/heads/master [following]
--2022-05-08 12:06:06--  https://codeload.github.com/paul2048/annotated-images/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]   5.89K  --.-KB/s    in 0.002s  

2022-05-08 12:06:06 (3.56 MB/s) - ‘master.zip’ saved [6027]

Archive:  /content/master.zip
aabb7fea70287f6c782d49374d898acc2ab1065a
   creating: annotated-images-master/
  inflating: a

##Prepare the data

In [ ]:
from google.colab import drive
import shutil

drive.mount(paths['DRIVE'])
shutil.unpack_archive(files['LOGODET_3K'], paths['DATASET'])

Mounted at /content/drive


In [ ]:
# Logo names used as classes for the model that we will build
selected_logos = ['Apple', 'ASUS']

In [ ]:
import xml.etree.ElementTree as ET
import annotated_images

# Delete the test and train sets of the previous split
trainset_path = os.path.join(paths['DATASET'], 'train')
testset_path = os.path.join(paths['DATASET'], 'test')
if os.path.isdir(trainset_path):
    shutil.rmtree(trainset_path)
if os.path.isdir(testset_path):
    shutil.rmtree(testset_path)

for category in os.listdir(paths['LOGODET_3K']):
    brands_path = os.path.join(paths['LOGODET_3K'], category)
    # Loop over the brands (7-Up, Apple, etc.)
    for brand in os.listdir(brands_path):
        if brand in selected_logos:
            brand_path = os.path.join(brands_path, brand)
            # Loop over each file (1.jpg, 1.xml, etc.)
            for file_ in os.listdir(brand_path):
                source_path = os.path.join(brand_path, file_)
                # Append the name of the brand at the beginning of the file name
                # because "1.jpg" (for example) repeats at each brand
                if file_.endswith('.xml'):
                    with open(source_path) as f:
                        tree = ET.parse(f)
                        root = tree.getroot()
                        xml_filename = root.find('filename')
                        num, extenssion = xml_filename.text.split('.')
                        num = num.split('_')[-1]
                        xml_filename.text = f'{brand}_{num}.{extenssion}'
                    tree.write(source_path)
                destination_path = os.path.join(paths['LOGODET_3K_FLAT_SUBSET'], f'{brand}_{file_}')
                shutil.copy(source_path, destination_path)

annotated_images.split(paths['LOGODET_3K_FLAT_SUBSET'], output_dir=paths['DATASET'], ratio=(0.7, 0.3))

{'test': 50, 'total': 165, 'train': 115}

## Download pre-trained model

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200713/{PRETRAINED_MODEL_NAME}.tar.gz
!tar -xvzf {PRETRAINED_MODEL_NAME}.tar.gz -C {ROOT_PATH}
!rm {PRETRAINED_MODEL_NAME}.tar.gz
# Rename the "checkpoint" directory (https://stackoverflow.com/a/64159833/7367049)
os.rename(
    os.path.join(paths['PRETRAINED_MODEL'], 'checkpoint'),
    os.path.join(paths['PRETRAINED_MODEL'], 'checkpoint0'))

--2022-05-08 12:59:50--  http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.152.128, 2607:f8b0:4001:c56::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.152.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1426099846 (1.3G) [application/x-tar]
Saving to: ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’

centernet_hg104_512 100%[===================>]   1.33G   135MB/s    in 9.1s    

2022-05-08 12:59:59 (149 MB/s) - ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’ saved [1426099846/1426099846]

centernet_hg104_512x512_coco17_tpu-8/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
centernet_hg104_512x512_coco17_tpu-8/checkpoint/checkpoint
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.index
centernet_hg104_512x512_coco17_tpu-8/p

## Create label map

In [ ]:
label_map = '\n'.join([
    f'item {{\n    id: {i+1}\n    name: "{brand_name}"\n}}'
    for i, brand_name in enumerate(selected_logos)
])
label_map

'item {\n    id: 1\n    name: "Apple"\n}\nitem {\n    id: 2\n    name: "ASUS"\n}'

In [ ]:
with open(files['LABEL_MAP'], 'w+') as f:
    f.write(label_map)

## Proto files

In [ ]:
# Protos list: https://github.com/tensorflow/models/tree/master/research/object_detection/protos
!cd {TFOD_DIR_NAME}/research && \
    protoc object_detection/protos/*.proto --python_out=. && \
    cp object_detection/packages/tf2/setup.py . && \
    python -m pip install .
import object_detection

Processing /content/tf_object_detection/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.2 MB 8.5 MB/s 
     |████████████████████████████████| 352 kB 47.5 MB/s 
     |████████████████████████████████| 2.2 MB 31.6 MB/s 
     |████████████████████████████████| 23.4 MB 1.8 MB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 
     |████████████████████████████████| 47.8 MB 50 kB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.5 MB/s 
     |████████████████████████████████| 237 kB 47.0 MB/s 
     |██████████████████████████████

In [ ]:
!wget https://github.com/sglvladi/TensorFlowObjectDetectionTutorial/raw/master/docs/source/scripts/generate_tfrecord.py
%cd {ROOT_PATH}
!python generate_tfrecord.py -x {paths['TRAIN_SET']} -l {files['LABEL_MAP']} -o {os.path.join(paths['ANNOTATIONS'], 'train.record')}
!python generate_tfrecord.py -x {paths['TEST_SET']} -l {files['LABEL_MAP']} -o {os.path.join(paths['ANNOTATIONS'], 'test.record')}

--2022-05-08 13:01:10--  https://github.com/sglvladi/TensorFlowObjectDetectionTutorial/raw/master/docs/source/scripts/generate_tfrecord.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sglvladi/TensorFlowObjectDetectionTutorial/master/docs/source/scripts/generate_tfrecord.py [following]
--2022-05-08 13:01:10--  https://raw.githubusercontent.com/sglvladi/TensorFlowObjectDetectionTutorial/master/docs/source/scripts/generate_tfrecord.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6410 (6.3K) [text/plain]
Saving to: ‘generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   6.26K

## Config file 

In [ ]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
import tensorflow as tf
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.center_net.num_classes = len(selected_logos)
pipeline_config.model.center_net.image_resizer.keep_aspect_ratio_resizer.min_dimension = 256
pipeline_config.model.center_net.image_resizer.keep_aspect_ratio_resizer.max_dimension = 256
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.max_number_of_boxes = 25
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL'], 'checkpoint0', 'ckpt-0')
pipeline_config.train_input_reader.label_map_path = files['LABEL_MAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATIONS'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABEL_MAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATIONS'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], 'wb') as f:
    f.write(config_text)

## Train the model

In [ ]:
# Fixes "ImportError: cannot import name '_registerMatType' from 'cv2.cv2'" when training
!pip install 'opencv-python-headless<4.3'

     |████████████████████████████████| 21.6 MB 19.7 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.64
    Uninstalling opencv-python-headless-4.5.5.64:
      Successfully uninstalled opencv-python-headless-4.5.5.64


In [ ]:
train_steps = 2000
command = f'python {files["TRAINING_SCRIPT"]} \
            --model_dir={paths["PRETRAINED_MODEL"]} \
            --pipeline_config_path="{files["PIPELINE_CONFIG"]}" \
            --checkpoint_every_n={train_steps} \
            --num_train_steps={train_steps}'
!{command}

In [ ]:
from tensorboard import notebook
%load_ext tensorboard

In [ ]:
%tensorboard --logdir {paths['PRETRAINED_MODEL']}/train

## Evaluate the model

In [ ]:
command = f'python {files["TRAINING_SCRIPT"]} \
            --model_dir={paths["PRETRAINED_MODEL"]} \
            --pipeline_config_path={files["PIPELINE_CONFIG"]} \
            --checkpoint_dir={paths["PRETRAINED_MODEL"]}'
!{command}

In [ ]:
%tensorboard --logdir {paths['PRETRAINED_MODEL']}/eval

## Load the trained model from checkpoint

In [28]:
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['PRETRAINED_MODEL'], 'ckpt-22')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## Test image

In [ ]:
import cv2 
import numpy as np
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABEL_MAP'])
IMAGE_PATH = '/content/30.jpg'

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes'] + label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()